In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import jaccard_score
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

 

In [2]:
anime=pd.read_csv(r"/Users/shreyasingh/Downloads/Recommendation System/anime.csv",encoding='Latin1')
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,GintamaÂ°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
anime.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [4]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [5]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [6]:
# Check for missing values  
anime['genre'] = anime['genre'].apply(lambda x: x if isinstance(x, list) else (x.split(', ') if isinstance(x, str) else []))  


In [7]:
# Handle missing values
anime['type'] = anime['type'].fillna('Unknown')
anime['rating'] = anime['rating'].fillna(anime['rating'].mean())
anime['episodes'] = pd.to_numeric(anime['episodes'], errors='coerce')
anime['episodes'] = anime['episodes'].fillna(anime['episodes'].median())

In [8]:
# Replace NaN values with an empty list  
anime['genre'] = anime['genre'].apply(lambda x: x if isinstance(x, list) else [])  

  

In [9]:
# Initialize MultiLabelBinarizer  
mlb = MultiLabelBinarizer()  

# Fit and transform the genre data  
genre_encoded = mlb.fit_transform(anime['genre']) 

In [10]:
# Normalize numerical features
scaler = MinMaxScaler()
numerical_features = scaler.fit_transform(anime[['anime_id', 'rating']])  

In [11]:
# Combine all features
feature_matrix = np.hstack([numerical_features, genre_encoded])

In [12]:
anime2=anime.iloc[:,1:]
anime2

,name,genre,type,episodes,rating,members
0,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1.0,9.37,200630
1,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64.0,9.26,793665
2,GintamaÂ°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51.0,9.25,114262
3,Steins;Gate,"[Sci-Fi, Thriller]",TV,24.0,9.17,673572
4,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51.0,9.16,151266
...,...,...,...,...,...,...
12289,Toushindai My Lover: Minami tai Mecha-Minami,[Hentai],OVA,1.0,4.15,211
12290,Under World,[Hentai],OVA,1.0,4.28,183
12291,Violence Gekiga David no Hoshi,[Hentai],OVA,4.0,4.88,219
12292,Violence Gekiga Shin David no Hoshi: Inma Dens...,[Hentai],OVA,1.0,4.98,175


In [13]:
anime2.sort_values(["genre"])
anime2

,name,genre,type,episodes,rating,members
0,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1.0,9.37,200630
1,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64.0,9.26,793665
2,GintamaÂ°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51.0,9.25,114262
3,Steins;Gate,"[Sci-Fi, Thriller]",TV,24.0,9.17,673572
4,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51.0,9.16,151266
...,...,...,...,...,...,...
12289,Toushindai My Lover: Minami tai Mecha-Minami,[Hentai],OVA,1.0,4.15,211
12290,Under World,[Hentai],OVA,1.0,4.28,183
12291,Violence Gekiga David no Hoshi,[Hentai],OVA,4.0,4.88,219
12292,Violence Gekiga Shin David no Hoshi: Inma Dens...,[Hentai],OVA,1.0,4.98,175


In [14]:
# Recommendation system function
def recommend_anime(target_anime, feature_matrix, anime_data, threshold=0.8):
    target_index = anime_data[anime_data['name'] == target_anime].index[0]
    target_features = feature_matrix[target_index]
    
    # Compute cosine similarity
    similarities = cosine_similarity([target_features], feature_matrix).flatten()
    
    # Get indices of similar anime above the threshold
    similar_indices = np.where(similarities > threshold)[0]
    
    # Exclude the target anime itself
    similar_indices = similar_indices[similar_indices != target_index]
    
    # Return the names of similar anime
    return anime_data.iloc[similar_indices][['name', 'rating', 'genre']]


In [15]:
# Example usage
recommended_anime = recommend_anime("Fullmetal Alchemist: Brotherhood", feature_matrix, anime, threshold=0.9)
print(recommended_anime)

                                               name  rating  \
200                             Fullmetal Alchemist    8.33   
402       Fullmetal Alchemist: Brotherhood Specials    8.11   
1558  Fullmetal Alchemist: The Sacred Star of Milos    7.50   

                                                  genre  
200   [Action, Adventure, Comedy, Drama, Fantasy, Ma...  
402   [Adventure, Drama, Fantasy, Magic, Military, S...  
1558  [Action, Adventure, Comedy, Drama, Fantasy, Ma...  


In [16]:
# Evaluation
# Split dataset into training and testing sets
train, test = train_test_split(anime, test_size=0.2, random_state=42)


In [ ]:
# Simplified evaluation: Use genres for precision, recall, and F1-score
true_labels = test['genre']
predicted_labels = []
for _, row in test.iterrows():
    recommendations = recommend_anime(row['name'],feature_matrix, anime, threshold=0.9)
    predicted_genres = set(recommendations['genre'].explode())
    true_genres = set(row['genre'])
    predicted_labels.append(1 if true_genres.intersection(predicted_genres) else 0)

In [ ]:
# Compute evaluation metrics
precision = precision_score([1] * len(test), predicted_labels, zero_division=1)
recall = recall_score([1] * len(test), predicted_labels, zero_division=1)
f1 = f1_score([1] * len(test), predicted_labels, zero_division=1)

print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

In [ ]:
genre_matrix = []

In [ ]:
# TF-IDF Vectorization for Genre Representation
tfidf = TfidfVectorizer(stop_words='english')
genre_matrix = tfidf.fit_transform(anime['genre'])

In [ ]:

# Content-based Recommendation (Cosine Similarity)
cos_sim = cosine_similarity(genre_matrix, genre_matrix)


In [ ]:
#using cosine similarity
def recommend_content_based(anime_id, top_n=5):
    idx = df[df['anime_id'] == anime_id].index[0]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    anime_indices = [i[0] for i in sim_scores]
    return df['name'].iloc[anime_indices]

In [ ]:
# Jaccard Similarity (For example, between two anime genres)
def jaccard_similarity(str1, str2):
    set1 = set(str1.split(', '))
    set2 = set(str2.split(', '))
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

In [ ]:
# Euclidean Distance (for genre-based feature vectors)
euclid_dist = euclidean_distances(genre_matrix)

# Example: Recommend anime using Euclidean distance
def recommend_euclidean(anime_id, top_n=5):
    idx = df[df['anime_id'] == anime_id].index[0]
    dist_scores = list(enumerate(euclid_dist[idx]))
    dist_scores = sorted(dist_scores, key=lambda x: x[1])
    dist_scores = dist_scores[1:top_n+1]
    anime_indices = [i[0] for i in dist_scores]
    return df['name'].iloc[anime_indices]

# Example usage of Euclidean distance-based recommendation
print("Euclidean Distance-Based Recommendations for anime_id 1:")
print(recommend_euclidean(1, top_n=5))

In [ ]:

# Visualizing the distribution of Jaccard similarity scores across genres
genre_similarities = []
for i in range(len(anime)):
    for j in range(i+1, len(anime)):
        genre_similarities.append(jaccard_similarity(anime['genre'][i], anime['genre'][j]))

# Plot Jaccard similarity distribution
plt.figure(figsize=(10, 6))
sns.histplot(genre_similarities, kde=True)
plt.title('Jaccard Similarity Distribution Between Anime Genres')
plt.xlabel('Jaccard Similarity')
plt.ylabel('Frequency')
plt.show()